# **IMPORT DATAFRAME DAN DATASET**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from google.colab import files
files = files.upload()

Saving dataset_clean.csv to dataset_clean.csv


In [ ]:
dataset = pd.read_csv("dataset_clean.csv")

In [ ]:
print('Shape dataset:', dataset.shape)
print(dataset)

Shape dataset: (877, 7)
     Unnamed: 0         HARGA     LB     LT    KT    KM  GRS
0             0  3.800000e+09  220.0  220.0   3.0   3.0  0.0
1             1  4.600000e+09  180.0  137.0   4.0   3.0  2.0
2             2  3.000000e+09  267.0  250.0   4.0   4.0  4.0
3             3  4.300000e+08   40.0   25.0   2.0   2.0  0.0
4             4  9.000000e+09  400.0  355.0   6.0   5.0  3.0
..          ...           ...    ...    ...   ...   ...  ...
872        1003  6.000000e+08   60.0   30.0   2.0   2.0  0.0
873        1004  2.600000e+09  100.0  140.0   3.0   2.0  1.0
874        1005  9.000000e+09  450.0  550.0  10.0  10.0  3.0
875        1006  4.000000e+09  160.0  140.0   4.0   3.0  2.0
876        1009  1.050000e+10  420.0  430.0   7.0   4.0  2.0

[877 rows x 7 columns]


# **MISSING VALUE DAN IMPUTASI DATA**

In [ ]:
print(dataset.isnull().sum())
#Menghitung total missing value
print(dataset.isnull().sum().sum())

Unnamed: 0    0
HARGA         0
LB            0
LT            0
KT            0
KM            0
GRS           0
dtype: int64
0


imputasi data fungsinya buat ngisi kolom yang NaN dengan sata-rata kelas . fungsi imputasi melakukan tahapan berikut:

1. Menghitung rata-rata masing-masing kolom berdasarkan kelasnya
2. Mengisi kolom yang berisi NaN dengan rata-rata kelas yang sesuai

In [ ]:
def imputasi(dataset_input) :
  list_columns = dataset_input.columns #mendapatkan daftar kolom pada dataframe
  class_column = list_columns[-1] #kolom terakhir merupakan kolom kelas
  for column in list_columns[:-1]:
    dataset_input[column] = dataset_input[column].fillna(dataset_input.groupby(class_column) [column].transform('mean'))
    #Penggantian nilai NaN dilakukan per kolom
  return dataset_input

In [ ]:
data_imputasi = imputasi (dataset)

In [ ]:
data_imputasi.isnull().values.any()

False

# **NORMALISASI DATA**

In [ ]:
#CEK RENTANG (NILAI MAX - NILAI MIN) MASING-MASING FITUR
def cetak_rentang(dataset_input) :
  list_fitur = dataset_input.columns[:-1] #mengambil nama kolom, kecuali yang terakhir (kelas)
  for fitur in list_fitur:
    max = dataset_input[fitur].max()
    min = dataset_input[fitur].min()
    print ("Rentang fitur",fitur,"adalah",max-min)

In [ ]:
cetak_rentang(data_imputasi)

Rentang fitur Unnamed: 0 adalah 1009
Rentang fitur HARGA adalah 17070000000.0
Rentang fitur LB adalah 760.0
Rentang fitur LT adalah 1200.0
Rentang fitur KT adalah 8.0
Rentang fitur KM adalah 9.0


Terlihat kalau masing-masing fitur memiliki rentang yang berbeda, Nah, fungsi normalisasi MinMax ini untuk menyamakan rentang setiap fitur menjadi satu 

In [ ]:
def minmax(dataset_input):
  list_fitur = dataset_input.columns[:-1]
  for fitur in list_fitur:
    max = dataset_input[fitur].max()
    min = dataset_input[fitur].min()
    dataset_input[fitur] = (dataset_input[fitur]-min)/(max-min)
  return dataset_input

In [ ]:
data_normal = minmax(data_imputasi)

In [ ]:
data_normal.head()

,Unnamed: 0,HARGA,LB,LT,KT,KM,GRS
0,0.000000,0.197422,0.236842,0.162500,0.125,0.222222,0.0
1,0.000991,0.244288,0.184211,0.093333,0.250,0.222222,2.0
2,0.001982,0.150557,0.298684,0.187500,0.250,0.333333,4.0
3,0.002973,0.000000,0.000000,0.000000,0.000,0.111111,0.0
4,0.003964,0.502050,0.473684,0.275000,0.500,0.444444,3.0


In [ ]:
cetak_rentang(data_normal)

Rentang fitur Unnamed: 0 adalah 1.0
Rentang fitur HARGA adalah 1.0
Rentang fitur LB adalah 1.0
Rentang fitur LT adalah 1.0
Rentang fitur KT adalah 1.0
Rentang fitur KM adalah 1.0


In [ ]:
harga = dataset["HARGA"]
luasbangunan = dataset["LB"]
luastanah = dataset["LT"]
kamartidur = dataset["KT"]
kamarmandi = dataset["KM"]
garasi = dataset["GRS"]

In [ ]:
df_all = []

for i in range(len(kamarmandi)):
  df = []
  df.append(luasbangunan[i])
  df.append(luastanah[i])
  df.append(kamartidur[i])
  df.append(kamarmandi[i])
  df.append(garasi[i])

  df_all.append(df)

In [ ]:
df_all = np.array(df_all)
harga = np.array(harga)

print(df_all)

[[0.23684211 0.1625     0.125      0.22222222 0.        ]
 [0.18421053 0.09333333 0.25       0.22222222 2.        ]
 [0.29868421 0.1875     0.25       0.33333333 4.        ]
 ...
 [0.53947368 0.4375     1.         1.         3.        ]
 [0.15789474 0.09583333 0.25       0.22222222 2.        ]
 [0.5        0.3375     0.625      0.33333333 2.        ]]


# **MEMBUAT MODEL**

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[5])])
optimize = Adam(lr=0.01, beta_1=0.9, beta_2=0.99)
model.compile(optimizer=optimize, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(
    #loss='mae',
    #optimizer='sgd',
    #metrics=['accuracy']
#)
model.fit(df_all, harga, epochs=500)

Epoch 1/500


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023 
Epoch 2/500
28/28 [==============================] - 0s 2ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 3/500
28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 4/500
28/28 [==============================] - 0s 2ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 5/500
28/28 [==============================] - 0s 2ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 6/500
28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 7/500
28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 8/500
28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 9/500
28/28 [==============================] - 0s 1ms/step - loss: 3.6814e-08 - accuracy: 0.0023
Epoch 10/500
28/28 [==============================] - 0s 2ms/step - loss: 3.6814e-08

In [ ]:
luas_bangunan = int(input("Luas Bangunan : "))
luas_tanah = int(input("Luas Tanah :"))
kamar_tidur = int(input("Jumlah Kamar Tidur : "))
kamar_mandi = int(input("Jumlah Kamar Mandi : "))
garasi = int(input("Jumlah Garasi : "))

Luas Bangunan : 20
Luas Tanah :20
Jumlah Kamar Tidur : 3
Jumlah Kamar Mandi : 2
Jumlah Garasi : 1


In [ ]:
prediksi_harga = model.predict([[luas_bangunan, luas_tanah, kamar_tidur, kamar_mandi, garasi]])
print ("Rumah Impian anda, kira-kira harganya {} Rupiah".format(prediksi_harga[0][0]))

Rumah Impian anda, kira-kira harganya 5.073782444000244 Rupiah
